In [3]:
from BRScraper import nba
import pandas as pd 
import numpy as np

In [5]:
stats_df = pd.DataFrame()
desired_seasons = ['2023-24', '2022-23', '2021-22'] # seasons to gather salaries for, will always include 1 season before the oldest listed

for season in desired_seasons:
    per_game_stats = nba.get_stats(int(season[:4]), info = 'per_game', rename = True)
    totals = nba.get_stats(int(season[:4]), info = 'totals', rename = True).drop(['Pos', 'Age', 'G', 'GS', 'Season'], axis = 1).reset_index(drop = True)
    advanced_stats = nba.get_stats(int(season[:4]), info = 'advanced', rename = True).drop(['Pos', 'Age', 'G', 'MP_advanced', 'Season'], axis = 1).reset_index(drop = True)

    nonfloat_cols = ['Player','Season','Pos','Age','Tm','G','GS']
    teams = per_game_stats.drop_duplicates(subset = ['Player'], keep = 'last').reset_index(drop = True)
    teams = teams['Tm']

    for column in per_game_stats.columns:
        if column not in nonfloat_cols:
            per_game_stats[column] = per_game_stats[column].astype(float)

    per_game_stats = per_game_stats.drop_duplicates(subset = ['Player'], keep = 'first').reset_index(drop = True)
    per_game_stats['Tm'] = teams

    for column in totals.columns:
        if column not in nonfloat_cols:
            totals[column] = totals[column].astype(float)

    totals = totals.drop_duplicates(subset = ['Player'], keep = 'first').reset_index(drop = True)
    totals['Tm'] = teams

    for column in advanced_stats.columns:
        if column not in nonfloat_cols:
            advanced_stats[column] = advanced_stats[column].astype(float)

    advanced_stats = advanced_stats.drop_duplicates(subset = ['Player'], keep = 'first').reset_index(drop = True)
    advanced_stats['Tm'] = teams
    
    stats = per_game_stats.merge(advanced_stats, on = ['Player', 'Tm'], how = 'left', validate = '1:1')
    stats = stats.merge (totals, on = ['Player', 'Tm'], how = 'left', validate = '1:1').fillna(0)

    stats = stats.astype({'G': int, 'GS': int, 'Age': int})

    stats['Player'] = stats['Player'].str.replace('.', '')
    stats['Player'] = stats['Player'].str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')
    stats['Player'] = stats['Player'].apply(lambda x: x.replace(' Jr', '') if x.endswith(' Jr') and x not in
        ['Jaren Jackson Jr', 'Tim Hardaway Jr', 'Gary Trent Jr', 'Larry Nance Jr', 
        'Duane Washington Jr', 'Scottie Pippen Jr', 'Vince Williams Jr', 'Ron Harper Jr'] else x)

    stats_df = pd.concat([stats_df, stats], ignore_index = True)

stats_df = stats_df.rename(columns={'Season':'Season S-1'})
stats_df.to_csv('stats.csv',sep = ',' , decimal = '.', index = False)
stats_df.head()

,Player,Pos,Age,Tm,G,GS,MP_per_game,FG_per_game,FGA_per_game,FG%_per_game,...,FT%_totals,ORB_totals,DRB_totals,TRB_totals,AST_totals,STL_totals,BLK_totals,TOV_totals,PF_totals,PTS_totals
0,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,0.485,...,0.702,100.0,228.0,328.0,50.0,31.0,30.0,59.0,102.0,508.0
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,...,0.364,214.0,271.0,485.0,97.0,36.0,46.0,79.0,98.0,361.0
2,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,0.540,...,0.806,184.0,504.0,688.0,240.0,88.0,61.0,187.0,208.0,1529.0
3,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,0.427,...,0.812,43.0,78.0,121.0,67.0,16.0,15.0,41.0,99.0,467.0
4,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,0.470,...,0.750,85.0,286.0,371.0,97.0,45.0,48.0,60.0,143.0,696.0
